In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx
import csv
from tqdm import tqdm

In [ ]:
def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

In [ ]:
def read_snapshot(filename):
    G = nx.Graph()
    with open(filename, 'r') as edgelistfile:
        data = csv.reader(edgelistfile)
        for row in data:
            G.add_node(int(row[0])) #superhero in first column
            G.add_node(int(row[1])) #superhero in second column
            G.add_edge(int(row[0]), int(row[1]), weight = 1)
        return G

def read_opinions(filename):
    colors = []
    with open(filename) as infile:
        opinions = infile.readlines()
    opinions = [float(op.strip()) for op in opinions]
    return opinions
    

def add_opinions(graph, opinions):
    for node in graph.nodes:
        graph.nodes[node]['opinion'] = opinions[node]

    return graph

def compute_ncc(graph):
    return nx.number_connected_components(graph)


In [ ]:
print("yay")

In [ ]:
import sys
sys.path.append("/home/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/netdspatch_local/")
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
import tqdm
import os
import numpy as np
from matplotlib.offsetbox import AnchoredText

warnings.filterwarnings("ignore")


def save_snapshot(model, its, modelname, name, nit):
    if not os.path.exists(f'snapshotGraphs/{modelname}/{name}/'):
        os.mkdir(f'snapshotGraphs/{modelname}/{name}/')

    fig, ax = plt.subplots(1, 1, num=1, figsize=(10, 8))
    G = model.graph.graph                    
    opinions = list(its['status'].values()) 
    pos = nx.spring_layout(G, seed=1)  # positions for all nodes
    nx.draw(G, pos, node_color=opinions, cmap=plt.cm.RdBu_r, node_size=300.0, vmin=0.0, vmax=1.0, width=0.1, alpha=1.0, ax = ax)
    plt.tight_layout()
    plt.savefig(f"snapshotGraphs/{modelname}/{name}/{name}_network {nit}.png")
    plt.close()

    sns.set_style("ticks")
    fig, ax = plt.subplots(1, 1, num=1, figsize=(10, 6))
    sns.despine()
    degrees = [G.degree(n) for n in G.nodes()]
    sns.histplot(x=degrees, kde=True)
    ax.grid(True)
    ax.tick_params(direction='out', length=10, width=3, colors = "black", labelsize=30, grid_color = "black", grid_alpha = 0.1)
    ax.set_ylabel("# Nodes", size=20)
    ax.set_xlabel("Degree", size=20)
    plt.savefig(f"snapshotGraphs/{modelname}/{name}/{name}_degreedist {nit}.png")
    plt.close()
    
    nx.write_edgelist(G, f"snapshotGraphs/{modelname}/{name}/{name}_edgelist {nit}.csv", delimiter=",")

    with open(f"snapshotGraphs/{modelname}/{name}/{name}_opinions {nit}.txt", "w") as opfile:
        for op in list(its['status'].values()):
            opfile.write(str(op)+"\n")
    
    C = nclusters(opinions, 0.001)
    
    if C == 1.0:
        return 1


def steady_state_coevolving(model, name, nsteady=1000, max_iterations=10, sensibility = 0.00001, node_status=True, progress_bar=True):
    system_status = []
    steady_it = 0
    for it in tqdm.tqdm(range(0, max_iterations), disable=not progress_bar):
        oldgraph = model.graph.graph.copy()
        its = model.iteration(node_status)
        save_snapshots_for = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100, 500, 1000, 2000, 5000, 10000, 100000, 500000, max_it]
        if it > 0:
            if np.all((its['max_diff'] < sensibility)):
                steady_it += 1
            else:
                steady_it = 0
        system_status.append(its)
        if oldgraph.edges() != model.graph.graph.edges() : steady_it = 0 
        if steady_it == nsteady:
            return system_status[:-nsteady]
        if it in save_snapshots_for:
            if save_snapshot(model, its, modelname, name, it) == 1:
                return system_status
    return system_status


In [ ]:
modelname = "rewiring"
n = 200
max_it = 1000000
initial_status = np.random.random_sample(n)
graphname = 'er'
p = 0.1
i = 1
graph = nx.erdos_renyi_graph(n, p, seed = i)
while nx.number_connected_components(graph) > 1:
    i+=1
    graph = nx.erdos_renyi_graph(n, p, seed = i)

In [ ]:
for pr in [0.0, 0.5, 1.0]:
    for e in [0.2]:
        for g in [0.0, 0.5, 1.0, 1.5]:
            name = f"{modelname} {graphname}{p} n{n} pr{pr} e{e} g{g} mi{max_it}"
            print(name)
            if modelname == "rewiring":
                model = op.AdaptiveAlgorithmicBiasModel(graph)
            else:
                model = op.AdaptivePeerPressureAlgorithmicBiasModel(graph)
            config = mc.Configuration()
            config.add_model_parameter("epsilon", e)
            config.add_model_parameter("gamma", g)
            config.add_model_parameter("p", pr)
            model.set_initial_status(config, opinions = initial_status)
            status = steady_state_coevolving(model, name, max_iterations=max_it+1, node_status=True, progress_bar=True)                

In [6]:
modelname = "triangles rewiring"
n = 200
max_it = 1000000
initial_status = np.random.random_sample(n)
graphname = 'er'
p = 0.1
i = 1
graph = nx.erdos_renyi_graph(n, p, seed = i)
while nx.number_connected_components(graph) > 1:
    i+=1
    graph = nx.erdos_renyi_graph(n, p, seed = i)
        
for pr in [0.0, 0.5, 1.0]:
    for e in [0.2]:
        for g in [0.0, 0.5, 1.0, 1.5]:
            name = f"{modelname} {graphname}{p} n{n} pr{pr} e{e} g{g} mi{max_it}"
            print(name)
            if modelname == "rewiring":
                model = op.AdaptiveAlgorithmicBiasModel(graph)
            else:
                model = op.AdaptivePeerPressureAlgorithmicBiasModel(graph)
            config = mc.Configuration()
            config.add_model_parameter("epsilon", e)
            config.add_model_parameter("gamma", g)
            config.add_model_parameter("p", pr)
            model.set_initial_status(config, opinions = initial_status)
            status = steady_state_coevolving(model, name, max_iterations=max_it+1, node_status=True, progress_bar=True)           

triangles rewiring er0.1 n200 pr0.0 e0.2 g0.0 mi1000000


  0%|          | 20/1000001 [00:04<61:27:43,  4.52it/s]


triangles rewiring er0.1 n200 pr0.0 e0.2 g0.5 mi1000000


  0%|          | 100/1000001 [00:06<16:55:55, 16.40it/s]


triangles rewiring er0.1 n200 pr0.0 e0.2 g1.0 mi1000000


  5%|▌         | 53778/1000001 [13:13<3:43:54, 70.43it/s]

In [ ]:
# from matplotlib import cm
# import numpy as np
# from matplotlib.colors import ListedColormap,LinearSegmentedColormap
# # top = cm.get_cmap('Reds_r', 128) # r means reversed version
# # center1 = cm.get_cmap('Greens', 128)
# # center2 = cm.get_cmap('Greens_r', 128)
# # bottom = cm.get_cmap('Blues', 128)# combine it all
# # newcolors = np.vstack((top(np.linspace(0, 1, 128)),
# #                         center1(np.linspace(0,1,128)),
# #                         center2(np.linspace(0,1,128)),
# #                        bottom(np.linspace(0, 1, 128))))# create a new colormaps with a name of OrangeBlue
# # mia = ListedColormap(newcolors, name='mia')

# # Red, Green, Blue
# # N = 256
# # vals = np.ones((N, 4))
# # # Red stays constant until middle of colormap all other channels increas
# # # to result in white
# # # from middle of colormap we decrease to 0, 0, 255 which is blue
# # vals[:, 0] = np.concatenate((np.linspace(1, 1, N//2), np.linspace(1, 0, N//2)), axis=None)
# # vals[:, 1] = np.concatenate((np.linspace(0, 1, N//2), np.linspace(1, 0, N//2)), axis=None)
# # vals[:, 2] = np.concatenate((np.linspace(0, 1, N//2), np.linspace(1, 1, N//2)), axis=None)
# # mia = ListedColormap(vals)

# colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # R -> G -> B
# n_bins = 100  # Discretizes the interpolation into bins
# cmap_name = 'mia'
# mia = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

In [ ]:
infos = []
models = ["rewiring"]
graphs = ["er","ba"]
pr = 0.5
epss = [0.2]
gams = [0.5]
mi = 100000
for model in models:
    for graph in graphs:
        if graph == "er":
            p = 0.1
        else:
            p = 5
        for eps in epss:
            for gam in gams:    
                dirname = f"snapshotGraphs/{model}/{model} {graph}{p} pr{pr} e{eps} g{gam} mi{mi}/"
                for nit in range(100000):
                    try:
                        filename = f"edgelist {nit}.csv"
                        opfile = 'opinions '+str(nit)+'.txt'
                        opinions, colors = read_opinions(os.path.join(dirname, opfile))
                        print("avg opinion is ", sum(opinions)/len(opinions))
                        C = nclusters(opinions, 0.01)
                        print("avg_ncluster is ", C)
                        g = read_snapshot(os.path.join(dirname, filename))
                        pos = nx.spring_layout(g, seed=1)  # positions for all nodes
                        g = add_opinions(g, opinions)
                        nx.draw(g, pos, node_color=opinions, cmap=plt.cm.RdBu, node_size=60.0, vmin=0.0, vmax=1.0, width=0.1, alpha=1.0)
                        plt.title(f"{model}_{graph}_{pr}_{eps}_{gam}_{nit}")
                        sm = plt.cm.ScalarMappable(cmap=plt.cm.RdBu, norm=plt.Normalize(vmin = 0, vmax=1))
                        sm._A = []
                        cbar = plt.colorbar(sm)
                        for t in cbar.ax.get_yticklabels():
                            t.set_fontsize(10)
                        cbar.outline.set_visible(False)
                        cbar.ax.tick_params()
                        plt.show()
                        plt.close()
                    except:
                        
                        continue
                    # nc = compute_ncc(g)
                    # d['nit'] = nit
                    # d['nc'] = nc
                    # infos.append(d)
                else:
                    continue
# import pandas as pd
# results = pd.DataFrame(infos)
# results.to_csv(f"connected_components_{model}.csv")
# %matplotlib inline
# for eps in [0.2, 0.3, 0.4]:
#     for gam in [0.0, 0.5, 1.0, 1.5]:
#         data = results[results['graph']=='er']
#         data2 = data[data['eps']==eps]
#         data3 = data2[data2['gam'] == gam]
#         sns.lineplot(data=data, x="nit", y="nc", marker = "s")
#         plt.title('_'.join([model, str(eps), str(gam)]))
#         plt.show()

In [ ]:
infos = []
models = ["triangles rewiring"]
graphs = ["er", "ba"]
pr = 0.5
epss = [0.2, 0.3, 0.4]
gams = [0.0, 0.5, 1.0, 1.5]
mi = 100000
for model in models:
    for graph in graphs:
        if graph == "er":
            p = 0.1
        else:
            p = 5
        for eps in epss:
            for gam in gams:    
                dirname = f"snapshotGraphs/{model}/{model} {graph}{p} pr{pr} e{eps} g{gam} mi{mi}/"
                for filename in os.listdir(f"{dirname}/"):
                    if filename.startswith('edgelist'):
                        d = {'model':model, 'graph':graph, 'p':p, 'pr':pr, 'eps':eps, 'gam':gam, 'mi':mi}
                        nit = int((filename.split(' ')[1]).split('.')[0])
                        opfile = 'opinions '+str(nit)+'.txt'
                        opinions, colors = read_opinions(os.path.join(dirname, opfile))
                        g = read_snapshot(os.path.join(dirname, filename))
                        g = add_opinions(g, opinions)
                        nc = compute_ncc(g)
                        d['nit'] = nit
                        d['nc'] = nc
                        infos.append(d)
                    else:
                        continue
import pandas as pd
results = pd.DataFrame(infos)
results.to_csv(f"connected_components_{model}.csv")

%matplotlib inline
for eps in [0.2, 0.3, 0.4]:
    for gam in [0.0, 0.5, 1.0, 1.5]:
        data = results[results['graph']=='er']
        data2 = data[data['eps']==eps]
        data3 = data2[data2['gam'] == gam]
        sns.lineplot(data=data, x="nit", y="nc", marker = "s")
        plt.title('_'.join([model, str(eps), str(gam)]))
        plt.show()

In [ ]:
models = ["rewiring"]
graphs = ["er", "ba"]
pr = 0.5
epss = [0.2, 0.3, 0.4]
gams = [0.0, 0.5, 1.0, 1.5]
mi = 100000

%matplotlib inline
for model in models:
    for graph in graphs:
        if graph == "er":
            p = 0.1
        else:
            p = 5
        for eps in epss:
            for gam in gams:    
                dirname = f"snapshotGraphs/{model}/{model} {graph}{p} pr{pr} e{eps} g{gam} mi{mi}/"
                for nit in range(100000):
                    try:
                        fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 7), dpi=600)
                        filename = f"edgelist {nit}.csv"
                        opfile = 'opinions '+str(nit)+'.txt'
                        opinions, colors = read_opinions(os.path.join(dirname, opfile))
                        print("avg opinion is ", sum(opinions)/len(opinions))
                        C = nclusters(opinions, 0.01)
                        print("avg_ncluster is ", C)
                        g = read_snapshot(os.path.join(dirname, filename))
                        pos = nx.spring_layout(g, seed=1)  # positions for all nodes
                        g = add_opinions(g, opinions)
                        nx.draw(g, pos, node_color=opinions, cmap=plt.cm.RdBu, node_size=60.0, vmin=0.0, vmax=1.0, width=0.1, alpha=1.0, ax=axes[0])
                        plot_degree_dist(g, ax=axes[1])
                        plt.sup_title(f"{model}_{graph}_{pr}_{eps}_{gam}_{nit}")
                        plt.show()
                    except:
                        
                        continue
                    # nc = compute_ncc(g)
                    # d['nit'] = nit
                    # d['nc'] = nc
                    # infos.append(d)
                else:
                    continue

